In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import tensorflow as tf
import warnings
import tensorflow_hub as hub
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

In [2]:
test_articles = pd.read_csv('../dataset/test_articles_ourdata.csv', delimiter=',')

In [3]:
fixed_index = pd.read_csv('../dataset/fixed_test_set_ourdata.csv', delimiter=',')

In [4]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [5]:
def get_mrr(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    index_match = 1

    for idTable in ranked_tables_ID:
        
        if idTable[0] == ID_goal:
    
            accuracy = 1/index_match
            break;
        
        index_match = index_match + 1
   
    return accuracy

In [6]:
test_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   article_id                948 non-null    int64 
 1   article_title             948 non-null    object
 2   article_full_text         948 non-null    object
 3   article_meta_description  948 non-null    object
 4   article_summary           948 non-null    object
 5   article_keywords          948 non-null    object
 6   article_meta_key_words    662 non-null    object
 7   article_tags              207 non-null    object
dtypes: int64(1), object(7)
memory usage: 59.4+ KB


In [7]:
fixed_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94721 entries, 0 to 94720
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label_index              94721 non-null  int64 
 1   table_page_id            94721 non-null  int64 
 2   table_page_title         94721 non-null  object
 3   table_page_main_passage  94721 non-null  object
 4   table_page_keywords      94721 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.6+ MB


In [8]:
evaluate_topk = [1,5,10,20,50]

result = []

#evaluating each topk value
for TOP_K in evaluate_topk:

    accuracy = []
    mrr = []

    for i, row in tqdm(test_articles.iterrows()):

        #current article values
        article_ID = row['article_id']
        article_title_text = str(row['article_title'])
        article_meta_description_text = str(row['article_meta_description'])
        article_keywords_text = str(row['article_keywords'])

        #embedding and model variables
        table_corpus1 = []
        table_corpus2 = []
        table_corpus3 = []
        ranked_tables_model = []

        #return index
        return_index = fixed_index.loc[fixed_index['label_index'] == row['article_id']]
        

        #creating embedding1 
        for i, row in return_index.iterrows():
            
            table_corpus1.append(str(row['table_page_title']))
              
        tokenized_corpus = [doc.split(" ") for doc in table_corpus1]
        bm25 = BM25Okapi(tokenized_corpus)
        
        tokenized_query = (article_title_text).split(" ")
        distance_vector1 = bm25.get_scores(tokenized_query)
        
        #creating embedding2 
        for i, row in return_index.iterrows():
            
            table_corpus2.append(str(row['table_page_main_passage']))
              
        tokenized_corpus = [doc.split(" ") for doc in table_corpus2]
        bm25 = BM25Okapi(tokenized_corpus)
        
        tokenized_query = (article_meta_description_text).split(" ")
        distance_vector2 = bm25.get_scores(tokenized_query)
        
        #creating embedding3 
        for i, row in return_index.iterrows():
            
            table_corpus3.append(str(row['table_page_keywords']))
              
        tokenized_corpus = [doc.split(" ") for doc in table_corpus3]
        bm25 = BM25Okapi(tokenized_corpus)
        
        tokenized_query = (article_keywords_text).split(" ")
        distance_vector3 = bm25.get_scores(tokenized_query)
        
        
        final_vector = (np.array(distance_vector1) + np.array(distance_vector2) + np.array(distance_vector3)) / 3.0
        
        
        
        #creating the final dataframe
        for i in range(0,len(final_vector)):

            ranked_tables_model.append([return_index.iloc[i]['table_page_id'],return_index.iloc[i]['table_page_title'],final_vector[i]]) 

        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking', ascending=False)
        
        selected_top = data_frame_sorting.head(TOP_K)
#         min_score = selected_top['table_ranking'].min()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] >= min_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values

        #getting topk accuracy
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))

        #testing mean reciprocal rank at k = 50
        if TOP_K == 50:

            mrr.append(get_mrr(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(TOP_K),str(round(np.mean(accuracy),4))])

print("BM25 METHOD")
print(result[0])
print(result[1])
print(result[2])
print(result[3])
print(result[4])
print("MRR: "+str(round(np.mean(mrr),4)) )
print(mrr)
print("")

948it [00:58, 16.22it/s]
948it [00:58, 16.24it/s]
948it [00:58, 16.17it/s]
948it [00:58, 16.25it/s]
948it [00:59, 16.02it/s]

BM25 METHOD
['Acc@1', '0.5158']
['Acc@5', '0.7627']
['Acc@10', '0.847']
['Acc@20', '0.9135']
['Acc@50', '0.9662']
MRR: 0.6272
[0.1111111111111111, 0.05, 1.0, 1.0, 0.3333333333333333, 0.5, 1.0, 1.0, 1.0, 1.0, 0.05263157894736842, 0.25, 0.047619047619047616, 1.0, 0.08333333333333333, 0.3333333333333333, 0.5, 0.5, 0.25, 0.5, 1.0, 0.2, 1.0, 0.16666666666666666, 0.07142857142857142, 0.25, 1.0, 0.5, 0, 0.25, 1.0, 0.5, 1.0, 0.05, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.05263157894736842, 0.1, 0.1111111111111111, 1.0, 0.09090909090909091, 1.0, 1.0, 1.0, 0.03125, 0, 1.0, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 1.0, 0.09090909090909091, 0.5, 1.0, 0, 1.0, 0.08333333333333333, 0.03125, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 1.0, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 0.25, 0.14285714285714285, 1.0, 1.0, 1.0, 1.0, 0.2, 1.0, 0, 0.3333333333333333, 0.029411764705882353, 0.5, 0.5, 1.0, 1.0, 0.16666666666666666, 0.5, 0.07692307692307693, 0.3333333333333333, 1.0, 0.0625, 1.0, 0.3333333333333333, 0.0625, 1.0, 